### 4.1 预测销售额原理

通过对数据的清理和对其创建特征，我们可以对这些特征进行提取并且创建我们的模型，从而我们可以对其有相同特征的时间来进行销售额的预测。

### 4.2

我们可以合理地预测每台售货机每个大类商品在2018年1月的交易额因为我们有2017年的完整销售数据，包括每月的交易额、售货机ID和商品大类。这些数据提供了足够的历史趋势和季节性信息，同时我们可以选择SARIMA模型进行时间序列分析因为SARIMA模型能够捕捉数据中的季节性和趋势，适合用于有时间依赖性的销售预测。

### 4.3

In [10]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [27]:
data

,订单号,设备ID,应付金额,实际金额,商品,支付时间,地点,状态,提现,大类,二级类,年份,月份
0,DD201708167493663618499909784,E43A6E078A07631,4.5,4.5,68g好丽友巧克力派2枚,2017-01-01 00:53:00,D,已出货未退款,已提现,非饮料,饼干糕点,2017,1
1,DD20170816749368329675770932,E43A6E078A07631,4.5,4.5,68g好丽友巧克力派2枚,2017-01-02 20:58:00,D,已出货未退款,已提现,非饮料,饼干糕点,2017,1
2,DD20170816749300229112037656709,E43A6E078A06874,4.5,4.5,68g好丽友巧克力派2枚,2017-01-03 01:53:00,E,已出货未退款,已提现,非饮料,饼干糕点,2017,1
3,DD201708167493529849068514902,E43A6E078A04228,4.0,4.0,68g好丽友巧克力派2枚,2017-01-08 20:22:00,C,已出货未退款,已提现,非饮料,饼干糕点,2017,1
4,DD201708167493876353091909391,E43A6E078A04228,14.0,14.0,68g好丽友巧克力派2枚,2017-01-09 21:38:00,C,已出货未退款,已提现,非饮料,饼干糕点,2017,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70675,DD201708167493878114605918673,E43A6E078A04228,11.0,11.0,20g洽洽每日坚果,2017-11-19 10:41:00,C,已出货未退款,已提现,非饮料,坚果炒货,2017,11
70676,DD201708167493690426013200961,E43A6E078A04228,4.0,4.0,商品36,2017-11-15 15:16:00,C,已出货未退款,已提现,非饮料,其他,2017,11
70677,DD201708167493774605081730209,E43A6E078A04172,11.0,11.0,80g粤光宝盒芒果干,2017-12-04 06:47:00,A,已出货未退款,已提现,非饮料,蜜饯/果干,2017,12
70678,DD2017041716541957D1BE9289A69,E43A6E078A04228,0.1,0.1,商品21,2017-12-10 16:18:00,C,已出货未退款,已提现,非饮料,其他,2017,12


In [21]:
data = pd.read_csv('merged_data.csv')

data['支付时间'] = pd.to_datetime(data['支付时间'], errors='coerce')

data = data.dropna(subset=['支付时间'])

data['年份'] = data['支付时间'].dt.year
data['月份'] = data['支付时间'].dt.month

data_2017 = data[(data['年份'] == 2017)]

monthly_sales = data_2017.groupby(['设备ID', '大类', '年份', '月份'])['实际金额'].sum().reset_index()

monthly_sales.head()


,设备ID,大类,年份,月份,实际金额
0,E43A6E078A04134,非饮料,2017,1,503.3
1,E43A6E078A04134,非饮料,2017,2,95.8
2,E43A6E078A04134,非饮料,2017,3,269.9
3,E43A6E078A04134,非饮料,2017,4,855.7
4,E43A6E078A04134,非饮料,2017,5,1730.6


In [26]:
def forecast_sales(data, device_id, category, forecast_periods=1):
    subset = data[(data['设备ID'] == device_id) & (data['大类'] == category)]
    subset = subset.set_index(pd.to_datetime(subset['年份'].astype(str) + '-' + subset['月份'].astype(str) + '-01'))
    subset = subset.sort_index()
    
    if len(subset) < 12:
        return None

    model = SARIMAX(subset['实际金额'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    results = model.fit(disp=False)

    forecast = results.get_forecast(steps=forecast_periods)
    return forecast.predicted_mean

forecast_results = []

device_ids = monthly_sales['设备ID'].unique()
categories = monthly_sales['大类'].unique()

for device_id in device_ids:
    for category in categories:
        prediction = forecast_sales(monthly_sales, device_id, category)
        if prediction is not None:
            forecast_results.append({
                '设备ID': device_id,
                '大类': category,
                '预测月份': '2018-01',
                '预测金额': prediction.iloc[0]
            })

forecast_df = pd.DataFrame(forecast_results)

/Users/aaron/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/aaron/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/aaron/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/Users/aaron/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations

In [25]:
forecast_df

,设备ID,大类,预测月份,预测金额
0,E43A6E078A04134,非饮料,2018-01,2510.250126
1,E43A6E078A04134,饮料,2018-01,6280.650218
2,E43A6E078A04172,非饮料,2018-01,3558.500198
3,E43A6E078A04172,饮料,2018-01,4783.450179
4,E43A6E078A04228,非饮料,2018-01,3132.850180
5,E43A6E078A04228,饮料,2018-01,7067.900230
6,E43A6E078A06874,非饮料,2018-01,4469.050114
7,E43A6E078A06874,饮料,2018-01,9916.850300
8,E43A6E078A07631,非饮料,2018-01,2212.100119
9,E43A6E078A07631,饮料,2018-01,4207.300120
